### Large Files Data Processing with DASK
- This was the original file we used with the intent of merging and cleaning the datasets.
- The file attempted to process teh entire datast using DASK but as the process went on the number of tasks which the dataframe had to compute grew so much that it put the kernel under too much stress so we had to re-evaluate our approach and integrate pandas, as shown in the other large file.
- We were able to get to the stage where we extracted the 46a merged file using DASK so it could be sue for tests.

In [1]:
import pandas as pd
import dask.dataframe as dd 
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt


In [2]:
!pip install matplotlib

In [3]:
cd tmp

/home/team9/tmp


In [4]:
ls

data/


In [5]:
cd data

/home/team9/tmp/data


In [6]:
ls

Custom_location_53_345035_-6_267261_62b5c8e6c91d98000ba01ceb.csv
rt_leavetimes_DB_2018.txt
rt_trips_DB_2018.txt
rt_vehicles_DB_2018.txt
rt_vehicles_DB_2018.zip
trips_modelling.csv
weather_factorised.csv
weather_trips.csv


In [7]:
weather = pd.read_csv('Custom_location_53_345035_-6_267261_62b5c8e6c91d98000ba01ceb.csv')

In [8]:
weather.shape

(27186, 28)

In [9]:
weather

,dt,dt_iso,timezone,city_name,lat,lon,temp,visibility,dew_point,feels_like,...,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1483228800,2017-01-01 00:00:00 +0000 UTC,0,Custom location,53.345035,-6.267261,5.39,9999.0,4.35,1.78,...,NaN,2.30,NaN,NaN,NaN,75,501,Rain,moderate rain,10n
1,1483232400,2017-01-01 01:00:00 +0000 UTC,0,Custom location,53.345035,-6.267261,5.39,9999.0,4.35,2.28,...,NaN,1.51,NaN,NaN,NaN,75,501,Rain,moderate rain,10n
2,1483236000,2017-01-01 02:00:00 +0000 UTC,0,Custom location,53.345035,-6.267261,5.39,9999.0,4.35,2.28,...,NaN,0.64,NaN,NaN,NaN,75,500,Rain,light rain,10n
3,1483239600,2017-01-01 03:00:00 +0000 UTC,0,Custom location,53.345035,-6.267261,4.39,9999.0,3.36,0.04,...,NaN,0.17,NaN,NaN,NaN,75,500,Rain,light rain,10n
4,1483243200,2017-01-01 04:00:00 +0000 UTC,0,Custom location,53.345035,-6.267261,4.39,9999.0,2.42,0.04,...,NaN,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27181,1577818800,2019-12-31 19:00:00 +0000 UTC,0,Custom location,53.345035,-6.267261,9.33,9999.0,6.42,9.33,...,2.68,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n
27182,1577822400,2019-12-31 20:00:00 +0000 UTC,0,Custom location,53.345035,-6.267261,8.81,9999.0,6.09,7.02,...,NaN,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n
27183,1577826000,2019-12-31 21:00:00 +0000 UTC,0,Custom location,53.345035,-6.267261,7.86,9999.0,5.33,7.86,...,1.79,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n
27184,1577829600,2019-12-31 22:00:00 +0000 UTC,0,Custom location,53.345035,-6.267261,7.49,9999.0,4.09,7.49,...,1.79,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n


In [10]:
weather.nunique()

dt                     26280
dt_iso                 26280
timezone                   2
city_name                  1
lat                        1
lon                        1
temp                    2009
visibility                45
dew_point               1929
feels_like              2550
temp_min                 672
temp_max                 919
pressure                  74
sea_level                  0
grnd_level                 0
humidity                  73
wind_speed               101
wind_deg                 358
wind_gust                 85
rain_1h                  281
rain_3h                    0
snow_1h                   49
snow_3h                    0
clouds_all                17
weather_id                27
weather_main              10
weather_description       29
weather_icon              17
dtype: int64

- most look unhelpful
- keeping to categorical data
- weather description, weather icon, temperature, visibility, wind speed, temperature
- will condense the categories down for better predictions
- i think i can ignore the timezone issue because the weather icon says whether it is day or night and it is only 1 hour difference and there is a timezone element in the dt iso column
- there are not an equal amount of datetimes as total columns so must remove duplicates
- weather icon dropoped after model testing

In [11]:
weather = weather.drop(["dt_iso", "temp_min", "weather_id", "clouds_all", "snow_3h", "snow_1h", "rain_3h", "rain_1h", "wind_gust", "wind_deg", "humidity", "grnd_level", "sea_level", "pressure", "temp_max", "feels_like", "dew_point", "lon", "lat", "city_name", "timezone"], axis=1)

In [12]:
weather

,dt,temp,visibility,wind_speed,weather_main,weather_description,weather_icon
0,1483228800,5.39,9999.0,5.10,Rain,moderate rain,10n
1,1483232400,5.39,9999.0,4.10,Rain,moderate rain,10n
2,1483236000,5.39,9999.0,4.10,Rain,light rain,10n
3,1483239600,4.39,9999.0,6.20,Rain,light rain,10n
4,1483243200,4.39,9999.0,6.20,Clouds,broken clouds,04n
...,...,...,...,...,...,...,...
27181,1577818800,9.33,9999.0,0.45,Clouds,broken clouds,04n
27182,1577822400,8.81,9999.0,3.10,Clouds,overcast clouds,04n
27183,1577826000,7.86,9999.0,0.45,Clouds,overcast clouds,04n
27184,1577829600,7.49,9999.0,0.45,Clouds,broken clouds,04n


In [13]:
print(weather["temp"].max())
print(weather["temp"].min())



26.92
-4.61


In [14]:
weather.dtypes

dt                       int64
temp                   float64
visibility             float64
wind_speed             float64
weather_main            object
weather_description     object
weather_icon            object
dtype: object

In [15]:
dupli_epoch = weather[weather.duplicated(['dt'])]


In [16]:
dupli_epoch

,dt,temp,visibility,wind_speed,weather_main,weather_description,weather_icon
135,1483711200,11.39,9000.0,6.70,Rain,light rain,10d
137,1483714800,11.39,9999.0,6.20,Rain,light rain,10d
139,1483718400,11.92,9999.0,6.70,Rain,light rain,10d
199,1483930800,9.39,9999.0,9.80,Drizzle,light intensity drizzle,09n
655,1485568800,6.00,9999.0,2.60,Rain,light rain,10n
...,...,...,...,...,...,...,...
27002,1577188800,7.48,9000.0,0.89,Rain,light rain,10d
27062,1577401200,8.62,8000.0,3.10,Rain,light rain,10n
27064,1577404800,8.43,8000.0,2.10,Rain,light rain,10n
27066,1577408400,8.70,5000.0,3.10,Rain,light rain,10n


In [17]:
dupliexamine_epoch = weather.loc[weather['dt'] == 1483711200]
print(dupliexamine_epoch)
dupliexamine_epoch = weather.loc[weather['dt'] == 1483718400]
print(dupliexamine_epoch)
dupliexamine_epoch = weather.loc[weather['dt'] == 1483711200]
print(dupliexamine_epoch)
dupliexamine_epoch = weather.loc[weather['dt'] == 1483930800]
print(dupliexamine_epoch)
dupliexamine_epoch = weather.loc[weather['dt'] == 1577404800]
print(dupliexamine_epoch)
dupliexamine_epoch = weather.loc[weather['dt'] == 1577412000]
print(dupliexamine_epoch)

             dt   temp  visibility  wind_speed weather_main  \
134  1483711200  11.39      9000.0         6.7      Drizzle   
135  1483711200  11.39      9000.0         6.7         Rain   

              weather_description weather_icon  
134  light intensity drizzle rain          09d  
135                    light rain          10d  
             dt   temp  visibility  wind_speed weather_main  \
138  1483718400  11.92      9999.0         6.7      Drizzle   
139  1483718400  11.92      9999.0         6.7         Rain   

              weather_description weather_icon  
138  light intensity drizzle rain          09d  
139                    light rain          10d  
             dt   temp  visibility  wind_speed weather_main  \
134  1483711200  11.39      9000.0         6.7      Drizzle   
135  1483711200  11.39      9000.0         6.7         Rain   

              weather_description weather_icon  
134  light intensity drizzle rain          09d  
135                    light rain     

- duplicates are all basically the same with slight variations in the final 2 columns so are not worth keeping in

In [18]:
weather = weather.drop_duplicates(subset=['dt'])
weather=weather.reset_index(drop=True)

In [19]:
weather.loc[130:140]
#check to see drop happnened
# 135 should be gone

,dt,temp,visibility,wind_speed,weather_main,weather_description,weather_icon
130,1483696800,10.44,8000.0,6.0,Rain,light rain,10d
131,1483700400,11.39,9999.0,4.6,Rain,light rain,10d
132,1483704000,11.39,9999.0,6.2,Clouds,broken clouds,04d
133,1483707600,11.39,9999.0,7.2,Rain,light rain,10d
134,1483711200,11.39,9000.0,6.7,Drizzle,light intensity drizzle rain,09d
135,1483714800,11.39,9999.0,6.2,Drizzle,light intensity drizzle rain,09d
136,1483718400,11.92,9999.0,6.7,Drizzle,light intensity drizzle rain,09d
137,1483722000,11.92,9999.0,6.2,Drizzle,light intensity drizzle,09n
138,1483725600,11.39,9999.0,6.2,Clouds,broken clouds,04n
139,1483729200,11.39,9999.0,5.7,Drizzle,light intensity drizzle,09n


In [20]:
weather.to_csv('./weather_trips.csv')

In [21]:
def round_int(x, base):
    return int(base * round(float(x)/base))

weather['temp'] = weather['temp'].apply(lambda x: round_int(x, base=5))


In [22]:
weather.head()

,dt,temp,visibility,wind_speed,weather_main,weather_description,weather_icon
0,1483228800,5,9999.0,5.1,Rain,moderate rain,10n
1,1483232400,5,9999.0,4.1,Rain,moderate rain,10n
2,1483236000,5,9999.0,4.1,Rain,light rain,10n
3,1483239600,5,9999.0,6.2,Rain,light rain,10n
4,1483243200,5,9999.0,6.2,Clouds,broken clouds,04n


In [23]:
weather['temp'] = pd.factorize(weather['temp'], sort=True)[0]

In [24]:
weather.head(10)

,dt,temp,visibility,wind_speed,weather_main,weather_description,weather_icon
0,1483228800,2,9999.0,5.1,Rain,moderate rain,10n
1,1483232400,2,9999.0,4.1,Rain,moderate rain,10n
2,1483236000,2,9999.0,4.1,Rain,light rain,10n
3,1483239600,2,9999.0,6.2,Rain,light rain,10n
4,1483243200,2,9999.0,6.2,Clouds,broken clouds,04n
5,1483246800,2,9999.0,6.2,Clouds,scattered clouds,03n
6,1483250400,1,9999.0,5.7,Clouds,few clouds,02n
7,1483254000,1,9999.0,6.7,Clouds,few clouds,02n
8,1483257600,1,9999.0,6.2,Clouds,few clouds,02n
9,1483261200,2,9999.0,5.7,Clouds,few clouds,02d


In [25]:
weather.isnull().sum()

dt                      0
temp                    0
visibility             45
wind_speed              0
weather_main            0
weather_description     0
weather_icon            0
dtype: int64

In [26]:
mean_vis = weather['visibility'].mean()
weather['visibility'] = weather['visibility'].fillna(mean_vis)

weather['visibility'].round(0).astype(int)

weather['visibility'] = weather['visibility'].apply(lambda x: round_int(x, base=1000))

In [27]:
weather.loc[1000:1030]

,dt,temp,visibility,wind_speed,weather_main,weather_description,weather_icon
1000,1486828800,2,10000,8.8,Rain,light rain,10d
1001,1486832400,2,10000,9.8,Rain,light rain,10d
1002,1486836000,2,10000,11.3,Rain,light rain,10n
1003,1486839600,2,10000,9.3,Rain,light rain,10n
1004,1486843200,2,10000,10.8,Rain,light rain,10n
1005,1486846800,2,8000,9.3,Rain,light rain,10n
1006,1486850400,2,10000,9.8,Rain,light rain,10n
1007,1486854000,2,10000,10.3,Rain,light rain,10n
1008,1486857600,2,10000,9.3,Rain,light rain,10n
1009,1486861200,2,10000,9.8,Rain,light rain,10n


In [28]:
weather['visibility'] = pd.factorize(weather['visibility'], sort=True)[0]

In [29]:
weather.head(3)

,dt,temp,visibility,wind_speed,weather_main,weather_description,weather_icon
0,1483228800,2,10,5.1,Rain,moderate rain,10n
1,1483232400,2,10,4.1,Rain,moderate rain,10n
2,1483236000,2,10,4.1,Rain,light rain,10n


In [30]:
weather.nunique()

dt                     26280
temp                       7
visibility                11
wind_speed               101
weather_main              10
weather_description       29
weather_icon              17
dtype: int64

In [31]:
max_wind = weather["wind_speed"].max()
min_wind = weather["wind_speed"].min()

print(max_wind)
print(min_wind)

22.1
0.0


In [32]:
weather["wind_speed"] = weather["wind_speed"].apply(lambda x: round_int(x, base=2))

In [33]:
weather["wind_speed"] = pd.factorize(weather["wind_speed"], sort=True)[0]

In [34]:
weather.head(10)

,dt,temp,visibility,wind_speed,weather_main,weather_description,weather_icon
0,1483228800,2,10,3,Rain,moderate rain,10n
1,1483232400,2,10,2,Rain,moderate rain,10n
2,1483236000,2,10,2,Rain,light rain,10n
3,1483239600,2,10,3,Rain,light rain,10n
4,1483243200,2,10,3,Clouds,broken clouds,04n
5,1483246800,2,10,3,Clouds,scattered clouds,03n
6,1483250400,1,10,3,Clouds,few clouds,02n
7,1483254000,1,10,3,Clouds,few clouds,02n
8,1483257600,1,10,3,Clouds,few clouds,02n
9,1483261200,2,10,3,Clouds,few clouds,02d


### saving weather as this for initial model.
- rest of it is a way to condense it so it will be manageable to add it to the leavetimes file which is muc larger and requires efficient use of memory.

In [35]:
# weather.to_csv('./weather_trips.csv') - forgot needed numbers for models

In [36]:
weather['weather_main'] = weather['weather_main'].apply(lambda x: 1 if (x == "Rain" or x == "Mist" or x == "Thunderstorm") else 0)

- dropping the weather icon and weather description
    - after model testing

In [37]:
weather = weather.drop(["weather_description", "weather_icon"], axis=1)

In [38]:
weather

,dt,temp,visibility,wind_speed,weather_main
0,1483228800,2,10,3,1
1,1483232400,2,10,2,1
2,1483236000,2,10,2,1
3,1483239600,2,10,3,1
4,1483243200,2,10,3,0
...,...,...,...,...,...
26275,1577818800,3,10,0,0
26276,1577822400,3,10,2,0
26277,1577826000,3,10,0,0
26278,1577829600,2,10,0,0


In [39]:
#weather["weather_description"] = pd.factorize(weather["weather_description"], sort=True)[0]
weather["weather_main"] = pd.factorize(weather["weather_main"], sort=True)[0]

In [40]:
#weather["weather_icon"] = pd.factorize(weather["weather_icon"], sort=True)[0]

In [41]:
weather.head(5)

,dt,temp,visibility,wind_speed,weather_main
0,1483228800,2,10,3,1
1,1483232400,2,10,2,1
2,1483236000,2,10,2,1
3,1483239600,2,10,3,1
4,1483243200,2,10,3,0


In [42]:
weather.dtypes

dt              int64
temp            int64
visibility      int64
wind_speed      int64
weather_main    int64
dtype: object

In [43]:
weather['date'] = pd.to_datetime(weather['dt'], unit='s')
weather

,dt,temp,visibility,wind_speed,weather_main,date
0,1483228800,2,10,3,1,2017-01-01 00:00:00
1,1483232400,2,10,2,1,2017-01-01 01:00:00
2,1483236000,2,10,2,1,2017-01-01 02:00:00
3,1483239600,2,10,3,1,2017-01-01 03:00:00
4,1483243200,2,10,3,0,2017-01-01 04:00:00
...,...,...,...,...,...,...
26275,1577818800,3,10,0,0,2019-12-31 19:00:00
26276,1577822400,3,10,2,0,2019-12-31 20:00:00
26277,1577826000,3,10,0,0,2019-12-31 21:00:00
26278,1577829600,2,10,0,0,2019-12-31 22:00:00


In [44]:
# only interested in weather from 2018
weather = weather.loc[weather['date']<'2019']

In [45]:
weather = weather.loc[weather['date']>'2018']

In [46]:
weather=weather.reset_index(drop=True)
weather

,dt,temp,visibility,wind_speed,weather_main,date
0,1514768400,2,10,6,1,2018-01-01 01:00:00
1,1514772000,2,10,6,0,2018-01-01 02:00:00
2,1514775600,2,10,6,0,2018-01-01 03:00:00
3,1514779200,2,10,6,0,2018-01-01 04:00:00
4,1514782800,2,10,5,0,2018-01-01 05:00:00
...,...,...,...,...,...,...
8754,1546282800,3,10,1,0,2018-12-31 19:00:00
8755,1546286400,3,10,0,0,2018-12-31 20:00:00
8756,1546290000,3,10,0,0,2018-12-31 21:00:00
8757,1546293600,3,10,1,0,2018-12-31 22:00:00


In [47]:
#weather.to_csv('./weather_factorised.csv')

# Bus Data

In [48]:
%%time
trips = pd.read_csv('rt_trips_DB_2018.txt', sep=";", chunksize=50000)

CPU times: user 1.75 ms, sys: 2.68 ms, total: 4.43 ms
Wall time: 29.6 ms


In [49]:
print(type(trips))

<class 'pandas.io.parsers.readers.TextFileReader'>


In [50]:
#for chunk in trips:
#    print(chunk.nunique())

In [51]:
# trips.head(5)

In [52]:
# trips.tail(3)

In [53]:
%%time
trips = dd.read_csv('rt_trips_DB_2018.txt', sep=";", dtype={'DATASOURCE': 'category', 'DAYOFSERVICE': 'category', 'TRIPID': 'int32', 'LINEID': 'category', 'ROUTEID': 'category', 'DIRECTION': 'category', 'PLANNEDTIME_ARR': 'float64', 'PLANNEDTIME_DEP': 'float64', 'ACTUALTIME_ARR': 'float64', 'ACTUALTIME_DEP': 'float64', 'BASIN': 'category', 'TENDERLOT': 'category', 'SUPPRESSED': 'category', 'JUSTIFICATIONID': 'string', 'LASTUPDATE': 'string', 'NOTE': 'string'})
#trips

CPU times: user 21.2 ms, sys: 728 µs, total: 21.9 ms
Wall time: 59.6 ms


In [54]:
print(type(trips))

<class 'dask.dataframe.core.DataFrame'>


In [55]:
trips.map_partitions(type).compute()

0    <class 'pandas.core.frame.DataFrame'>
1    <class 'pandas.core.frame.DataFrame'>
2    <class 'pandas.core.frame.DataFrame'>
dtype: object

In [56]:
trips

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
npartitions=3,,,,,,,,,,,,,,,,
,category[unknown],category[unknown],int32,category[unknown],category[unknown],category[unknown],float64,float64,float64,float64,category[unknown],category[unknown],category[unknown],string,string,string
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [57]:
trips.head(3)

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,07-FEB-18 00:00:00,6253783,68,68_80,1,87245.0,84600.0,87524.0,84600.0,BasDef,NaN,NaN,<NA>,28-FEB-18 12:05:11,",2967409,"
1,DB,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517.0,26460.0,32752.0,NaN,BasDef,NaN,NaN,<NA>,28-FEB-18 12:05:11,",2580260,"
2,DB,07-FEB-18 00:00:00,6254942,45A,45A_70,2,35512.0,32100.0,36329.0,32082.0,BasDef,NaN,NaN,<NA>,28-FEB-18 12:05:11,",2448968,"


In [58]:
trips.tail(3)

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
727496,DB,14-MAY-18 00:00:00,6765486,33D,33D_62,2,29460.0,26400.0,29904.0,NaN,BasDef,NaN,NaN,<NA>,26-JUN-18 09:13:13,",3077688,"
727497,DB,14-MAY-18 00:00:00,6764987,70,70_60,1,65277.0,60600.0,66341.0,NaN,BasDef,NaN,NaN,<NA>,26-JUN-18 09:13:13,",3208841,"
727498,DB,14-MAY-18 00:00:00,6765012,27,27_19,1,47722.0,41700.0,47508.0,41642.0,BasDef,NaN,NaN,<NA>,26-JUN-18 09:13:13,",2960092,"


- multiple empty columns
- multiple probably useless ones
    - "note", "last update", "basin", "LASTUPDATE"
    

In [59]:
trips = trips.drop({'TENDERLOT', "SUPPRESSED", "JUSTIFICATIONID", "DATASOURCE", "LASTUPDATE", "NOTE","BASIN", "ACTUALTIME_ARR", "PLANNEDTIME_ARR"}, axis=1)


In [60]:
trips

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_DEP,ACTUALTIME_DEP
npartitions=3,,,,,,,
,category[unknown],int32,category[unknown],category[unknown],category[unknown],float64,float64
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [61]:
trips.compute()

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_DEP,ACTUALTIME_DEP
0,07-FEB-18 00:00:00,6253783,68,68_80,1,84600.0,84600.0
1,07-FEB-18 00:00:00,6262138,25B,25B_271,2,26460.0,NaN
2,07-FEB-18 00:00:00,6254942,45A,45A_70,2,32100.0,32082.0
3,07-FEB-18 00:00:00,6259460,25A,25A_273,1,54420.0,54443.0
4,07-FEB-18 00:00:00,6253175,14,14_15,1,81600.0,81608.0
...,...,...,...,...,...,...,...
727494,14-MAY-18 00:00:00,6765849,123,123_36,2,57840.0,57859.0
727495,14-MAY-18 00:00:00,6765469,75,75_17,1,48600.0,48823.0
727496,14-MAY-18 00:00:00,6765486,33D,33D_62,2,26400.0,NaN
727497,14-MAY-18 00:00:00,6764987,70,70_60,1,60600.0,NaN


In [63]:
trips

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_DEP,ACTUALTIME_DEP
npartitions=3,,,,,,,
,category[unknown],int32,category[unknown],category[unknown],category[unknown],float64,float64
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [64]:
trips.head(3)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_DEP,ACTUALTIME_DEP
0,07-FEB-18 00:00:00,6253783,68,68_80,1,84600.0,84600.0
1,07-FEB-18 00:00:00,6262138,25B,25B_271,2,26460.0,NaN
2,07-FEB-18 00:00:00,6254942,45A,45A_70,2,32100.0,32082.0


In [65]:
trips

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_DEP,ACTUALTIME_DEP
npartitions=3,,,,,,,
,category[unknown],int32,category[unknown],category[unknown],category[unknown],float64,float64
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [66]:
trips.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 7 entries, DAYOFSERVICE to ACTUALTIME_DEP
dtypes: category(4), float64(2), int32(1)

In [67]:
trips.describe()

,TRIPID,PLANNEDTIME_DEP,ACTUALTIME_DEP
npartitions=1,,,
,float64,float64,float64
,...,...,...


In [68]:
trips.drop({"PLANNEDTIME_DEP", "ACTUALTIME_DEP"}, axis=1).compute()


,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION
0,07-FEB-18 00:00:00,6253783,68,68_80,1
1,07-FEB-18 00:00:00,6262138,25B,25B_271,2
2,07-FEB-18 00:00:00,6254942,45A,45A_70,2
3,07-FEB-18 00:00:00,6259460,25A,25A_273,1
4,07-FEB-18 00:00:00,6253175,14,14_15,1
...,...,...,...,...,...
727494,14-MAY-18 00:00:00,6765849,123,123_36,2
727495,14-MAY-18 00:00:00,6765469,75,75_17,1
727496,14-MAY-18 00:00:00,6765486,33D,33D_62,2
727497,14-MAY-18 00:00:00,6764987,70,70_60,1


In [69]:
print(type(trips))

<class 'dask.dataframe.core.DataFrame'>


In [67]:
trips

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_DEP,ACTUALTIME_DEP
npartitions=3,,,,,,,
,category[unknown],int32,category[unknown],category[unknown],category[unknown],float64,float64
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [68]:
trips.head(2)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_DEP,ACTUALTIME_DEP
0,07-FEB-18 00:00:00,6253783,68,68_80,1,84600.0,84600.0
1,07-FEB-18 00:00:00,6262138,25B,25B_271,2,26460.0,NaN


In [69]:
#%%time
#leaveTimes = pd.read_csv('rt_leavetimes_DB_2018.txt', sep=";", chunksize=500000)

In [70]:
#%%time
#for chunk in leaveTimes:
#    print(chunk.nunique())

In [71]:
#%%time
#leaveTimes = pd.read_csv('rt_leavetimes_DB_2018.txt', sep=";", nrows=10)

In [72]:
#leaveTimes.head(5)

In [93]:
%%time
leaveTimes = dd.read_csv('rt_leavetimes_DB_2018.txt', sep=";", dtype={'DATASOURCE': 'category', 'DAYOFSERVICE': 'category', 'PROGRNUMBER': 'int8', 'STOPPOINTID': 'int32','TRIPID': 'int32', 'LINEID': 'category', 'ROUTEID': 'category', 'VEHICLEID': 'category', 'PLANNEDTIME_ARR': 'float64', 'PLANNEDTIME_DEP': 'float64', 'ACTUALTIME_ARR': 'float64', 'ACTUALTIME_DEP': 'float64','PASSENGERSIN': 'category', 'PASSENGERSOUT': 'category', 'DISTANCE': 'category', 'SUPPRESSED': 'category', 'JUSTIFICATIONID': 'string', 'LASTUPDATE': 'string', 'NOTE': 'string'})

CPU times: user 23.6 ms, sys: 4.94 ms, total: 28.6 ms
Wall time: 53.5 ms


In [94]:
leaveTimes.head(3)

,DATASOURCE,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,PASSENGERS,PASSENGERSIN,PASSENGERSOUT,DISTANCE,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,01-JAN-18 00:00:00,5972116,12,119,48030.0,48030.0,48012.0,48012.0,2693211,NaN,NaN,NaN,NaN,NaN,<NA>,08-JAN-18 17:21:10,<NA>
1,DB,01-JAN-18 00:00:00,5966674,12,119,54001.0,54001.0,54023.0,54023.0,2693267,NaN,NaN,NaN,NaN,NaN,<NA>,08-JAN-18 17:21:10,<NA>
2,DB,01-JAN-18 00:00:00,5959105,12,119,60001.0,60001.0,59955.0,59955.0,2693263,NaN,NaN,NaN,NaN,NaN,<NA>,08-JAN-18 17:21:10,<NA>


In [95]:
leaveTimes["STOPPOINTID"].max()

dd.Scalar<series-..., dtype=int32>

In [96]:
leaveTimes = leaveTimes.drop({"SUPPRESSED", "JUSTIFICATIONID", "DATASOURCE", "NOTE", "PASSENGERSOUT", "PASSENGERSIN", "PASSENGERS", "LASTUPDATE" }, axis=1)

In [97]:
leaveTimes.head(3)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,DISTANCE
0,01-JAN-18 00:00:00,5972116,12,119,48030.0,48030.0,48012.0,48012.0,2693211,NaN
1,01-JAN-18 00:00:00,5966674,12,119,54001.0,54001.0,54023.0,54023.0,2693267,NaN
2,01-JAN-18 00:00:00,5959105,12,119,60001.0,60001.0,59955.0,59955.0,2693263,NaN


In [98]:
leaveTimes.tail(3)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,DISTANCE
664448,31-DEC-18 00:00:00,8586183,78,4383,51481.0,51481.0,52237.0,52283.0,2693229,NaN
664449,31-DEC-18 00:00:00,8589374,23,7053,53659.0,53659.0,53525.0,53525.0,3265669,NaN
664450,31-DEC-18 00:00:00,8589372,24,2088,46383.0,46383.0,46315.0,46325.0,3265669,NaN


In [99]:
leaveTimes

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,DISTANCE
npartitions=176,,,,,,,,,,
,category[unknown],int32,int8,int32,float64,float64,float64,float64,category[unknown],category[unknown]
,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


In [111]:
# drops distance and switches to pandas
leaveTimes = leaveTimes.drop({"DISTANCE"}, axis=1).compute()


KeyboardInterrupt



In [102]:
leaveTimes

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID
0,01-JAN-18 00:00:00,5972116,12,119,48030.0,48030.0,48012.0,48012.0,2693211
1,01-JAN-18 00:00:00,5966674,12,119,54001.0,54001.0,54023.0,54023.0,2693267
2,01-JAN-18 00:00:00,5959105,12,119,60001.0,60001.0,59955.0,59955.0,2693263
3,01-JAN-18 00:00:00,5966888,12,119,58801.0,58801.0,58771.0,58771.0,2693284
4,01-JAN-18 00:00:00,5965960,12,119,56401.0,56401.0,56309.0,56323.0,2693209
...,...,...,...,...,...,...,...,...,...
664446,31-DEC-18 00:00:00,8588153,78,4383,28605.0,28605.0,28998.0,29013.0,3265721
664447,31-DEC-18 00:00:00,8587459,78,4383,22695.0,22695.0,23247.0,23247.0,3265687
664448,31-DEC-18 00:00:00,8586183,78,4383,51481.0,51481.0,52237.0,52283.0,2693229
664449,31-DEC-18 00:00:00,8589374,23,7053,53659.0,53659.0,53525.0,53525.0,3265669


In [103]:
len(leaveTimes)

116949113

In [104]:
leaveTimes.head(1)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID
0,01-JAN-18 00:00:00,5972116,12,119,48030.0,48030.0,48012.0,48012.0,2693211


In [105]:
#leaveTimes= leaveTimes.persist()

In [106]:
leaveTimes.tail()

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID
664446,31-DEC-18 00:00:00,8588153,78,4383,28605.0,28605.0,28998.0,29013.0,3265721
664447,31-DEC-18 00:00:00,8587459,78,4383,22695.0,22695.0,23247.0,23247.0,3265687
664448,31-DEC-18 00:00:00,8586183,78,4383,51481.0,51481.0,52237.0,52283.0,2693229
664449,31-DEC-18 00:00:00,8589374,23,7053,53659.0,53659.0,53525.0,53525.0,3265669
664450,31-DEC-18 00:00:00,8589372,24,2088,46383.0,46383.0,46315.0,46325.0,3265669


In [107]:
print(leaveTimes.reset_index(drop=True).tail())

                 DAYOFSERVICE   TRIPID  PROGRNUMBER  STOPPOINTID  \
116949108  31-DEC-18 00:00:00  8588153           78         4383   
116949109  31-DEC-18 00:00:00  8587459           78         4383   
116949110  31-DEC-18 00:00:00  8586183           78         4383   
116949111  31-DEC-18 00:00:00  8589374           23         7053   
116949112  31-DEC-18 00:00:00  8589372           24         2088   

           PLANNEDTIME_ARR  PLANNEDTIME_DEP  ACTUALTIME_ARR  ACTUALTIME_DEP  \
116949108          28605.0          28605.0         28998.0         29013.0   
116949109          22695.0          22695.0         23247.0         23247.0   
116949110          51481.0          51481.0         52237.0         52283.0   
116949111          53659.0          53659.0         53525.0         53525.0   
116949112          46383.0          46383.0         46315.0         46325.0   

          VEHICLEID  
116949108   3265721  
116949109   3265687  
116949110   2693229  
116949111   3265669  
116949

In [108]:
leaveTimes = leaveTimes.reset_index(drop=True)

In [109]:
leaveTimes

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID
0,01-JAN-18 00:00:00,5972116,12,119,48030.0,48030.0,48012.0,48012.0,2693211
1,01-JAN-18 00:00:00,5966674,12,119,54001.0,54001.0,54023.0,54023.0,2693267
2,01-JAN-18 00:00:00,5959105,12,119,60001.0,60001.0,59955.0,59955.0,2693263
3,01-JAN-18 00:00:00,5966888,12,119,58801.0,58801.0,58771.0,58771.0,2693284
4,01-JAN-18 00:00:00,5965960,12,119,56401.0,56401.0,56309.0,56323.0,2693209
...,...,...,...,...,...,...,...,...,...
116949108,31-DEC-18 00:00:00,8588153,78,4383,28605.0,28605.0,28998.0,29013.0,3265721
116949109,31-DEC-18 00:00:00,8587459,78,4383,22695.0,22695.0,23247.0,23247.0,3265687
116949110,31-DEC-18 00:00:00,8586183,78,4383,51481.0,51481.0,52237.0,52283.0,2693229
116949111,31-DEC-18 00:00:00,8589374,23,7053,53659.0,53659.0,53525.0,53525.0,3265669


- data is intact

In [110]:
leaveTimes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116949113 entries, 0 to 116949112
Data columns (total 9 columns):
 #   Column           Dtype   
---  ------           -----   
 0   DAYOFSERVICE     category
 1   TRIPID           int32   
 2   PROGRNUMBER      int8    
 3   STOPPOINTID      int32   
 4   PLANNEDTIME_ARR  float64 
 5   PLANNEDTIME_DEP  float64 
 6   ACTUALTIME_ARR   float64 
 7   ACTUALTIME_DEP   float64 
 8   VEHICLEID        category
dtypes: category(2), float64(4), int32(2), int8(1)
memory usage: 4.9 GB


In [85]:
#leaveTimes = leaveTimes.persist()

AttributeError: 'DataFrame' object has no attribute 'persist'

In [112]:
print(type(leaveTimes))

<class 'pandas.core.frame.DataFrame'>


In [113]:
%%time
leaveTimes

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 11 µs


,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID
0,01-JAN-18 00:00:00,5972116,12,119,48030.0,48030.0,48012.0,48012.0,2693211
1,01-JAN-18 00:00:00,5966674,12,119,54001.0,54001.0,54023.0,54023.0,2693267
2,01-JAN-18 00:00:00,5959105,12,119,60001.0,60001.0,59955.0,59955.0,2693263
3,01-JAN-18 00:00:00,5966888,12,119,58801.0,58801.0,58771.0,58771.0,2693284
4,01-JAN-18 00:00:00,5965960,12,119,56401.0,56401.0,56309.0,56323.0,2693209
...,...,...,...,...,...,...,...,...,...
116949108,31-DEC-18 00:00:00,8588153,78,4383,28605.0,28605.0,28998.0,29013.0,3265721
116949109,31-DEC-18 00:00:00,8587459,78,4383,22695.0,22695.0,23247.0,23247.0,3265687
116949110,31-DEC-18 00:00:00,8586183,78,4383,51481.0,51481.0,52237.0,52283.0,2693229
116949111,31-DEC-18 00:00:00,8589374,23,7053,53659.0,53659.0,53525.0,53525.0,3265669


In [241]:
#%%time
#vehicles = pd.read_csv('rt_vehicles_DB_2018.txt', sep=";", chunksize=50000)

CPU times: user 3.58 ms, sys: 3.09 ms, total: 6.67 ms
Wall time: 40.6 ms


In [242]:
#for chunk in vehicles:
 #   print(chunk.nunique())

DATASOURCE          1
DAYOFSERVICE      118
VEHICLEID        1090
DISTANCE        45302
MINUTES         29651
LASTUPDATE        118
NOTE                0
dtype: int64
DATASOURCE          1
DAYOFSERVICE       82
VEHICLEID        1072
DISTANCE        45363
MINUTES         30012
LASTUPDATE         82
NOTE                0
dtype: int64
DATASOURCE          1
DAYOFSERVICE       88
VEHICLEID        1045
DISTANCE        45379
MINUTES         30237
LASTUPDATE         88
NOTE                0
dtype: int64
DATASOURCE          1
DAYOFSERVICE       84
VEHICLEID        1097
DISTANCE        45353
MINUTES         30474
LASTUPDATE         84
NOTE                0
dtype: int64
DATASOURCE          1
DAYOFSERVICE       82
VEHICLEID        1042
DISTANCE        45349
MINUTES         30126
LASTUPDATE         82
NOTE                0
dtype: int64
DATASOURCE          1
DAYOFSERVICE       42
VEHICLEID        1064
DISTANCE        21607
MINUTES         17842
LASTUPDATE         42
NOTE                0
dtype: int6

In [243]:
#vehicles_p = pd.read_csv('rt_vehicles_DB_2018.txt', sep=";", nrows=100)

In [244]:
#vehicles_p.head(14)

,DATASOURCE,DAYOFSERVICE,VEHICLEID,DISTANCE,MINUTES,LASTUPDATE,NOTE
0,DB,23-NOV-18 00:00:00,3303848,286166,58849,04-DEC-18 08:03:09,NaN
1,DB,23-NOV-18 00:00:00,3303847,259545,56828,04-DEC-18 08:03:09,NaN
2,DB,28-FEB-18 00:00:00,2868329,103096,40967,08-MAR-18 10:35:59,NaN
3,DB,28-FEB-18 00:00:00,2868330,147277,43599,08-MAR-18 10:35:59,NaN
4,DB,28-FEB-18 00:00:00,2868331,224682,40447,08-MAR-18 10:35:59,NaN
5,DB,28-FEB-18 00:00:00,2868332,19499,6289,08-MAR-18 10:35:59,NaN
6,DB,28-FEB-18 00:00:00,2868333,133014,43647,08-MAR-18 10:35:59,NaN
7,DB,28-FEB-18 00:00:00,2868334,168964,47167,08-MAR-18 10:35:59,NaN
8,DB,28-FEB-18 00:00:00,2868335,360842,55477,08-MAR-18 10:35:59,NaN
9,DB,28-FEB-18 00:00:00,2868336,139959,39599,08-MAR-18 10:35:59,NaN


In [245]:
# vehicles_p.tail(3)

,DATASOURCE,DAYOFSERVICE,VEHICLEID,DISTANCE,MINUTES,LASTUPDATE,NOTE
97,DB,22-JUN-18 00:00:00,2868343,251644,45142,02-JUL-18 13:32:00,NaN
98,DB,22-JUN-18 00:00:00,2868344,198033,51354,02-JUL-18 13:32:00,NaN
99,DB,22-JUN-18 00:00:00,2868345,220367,59893,02-JUL-18 13:32:00,NaN


In [88]:
#%%time
#vehicles = dd.read_csv('rt_vehicles_DB_2018.txt', sep=";", dtype={'DATASOURCE': 'category', 'DAYOFSERVICE': 'string', 'VEHICLEID': 'int32', 'DISTANCE': 'int32','MINUTES': 'int32', "LASTUPDATE": "string", "NOTE": "string"})

CPU times: user 10.4 ms, sys: 6.63 ms, total: 17 ms
Wall time: 289 ms


In [247]:
#vehicles

,DATASOURCE,DAYOFSERVICE,VEHICLEID,DISTANCE,MINUTES,LASTUPDATE,NOTE
npartitions=1,,,,,,,
,category[unknown],string,int32,int32,int32,string,string
,...,...,...,...,...,...,...


In [248]:
#vehicles.head(3)

,DATASOURCE,DAYOFSERVICE,VEHICLEID,DISTANCE,MINUTES,LASTUPDATE,NOTE
0,DB,23-NOV-18 00:00:00,3303848,286166,58849,04-DEC-18 08:03:09,<NA>
1,DB,23-NOV-18 00:00:00,3303847,259545,56828,04-DEC-18 08:03:09,<NA>
2,DB,28-FEB-18 00:00:00,2868329,103096,40967,08-MAR-18 10:35:59,<NA>


In [249]:
#help(vehicles)

Help on DataFrame in module dask.dataframe.core object:

class DataFrame(_Frame)
 |  DataFrame(dsk, name, meta, divisions)
 |  
 |  Parallel Pandas DataFrame
 |  
 |  Do not use this class directly.  Instead use functions like
 |  ``dd.read_csv``, ``dd.read_parquet``, or ``dd.from_pandas``.
 |  
 |  Parameters
 |  ----------
 |  dsk: dict
 |      The dask graph to compute this DataFrame
 |  name: str
 |      The key prefix that specifies which keys in the dask comprise this
 |      particular DataFrame
 |  meta: pandas.DataFrame
 |      An empty ``pandas.DataFrame`` with names, dtypes, and index matching
 |      the expected output.
 |  divisions: tuple of index values
 |      Values along which we partition our blocks on the index
 |  
 |  Method resolution order:
 |      DataFrame
 |      _Frame
 |      dask.base.DaskMethodsMixin
 |      dask.utils.OperatorMethodMixin
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __array_wrap__(self, array, context=None)
 |  
 |  __

In [250]:
#vehicles.drop({"NOTE", "DATASOURCE"}, axis=1).compute()
#vehicles.head(2)

,DATASOURCE,DAYOFSERVICE,VEHICLEID,DISTANCE,MINUTES,LASTUPDATE,NOTE
0,DB,23-NOV-18 00:00:00,3303848,286166,58849,04-DEC-18 08:03:09,<NA>
1,DB,23-NOV-18 00:00:00,3303847,259545,56828,04-DEC-18 08:03:09,<NA>


In [251]:
#vehicles.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 7 entries, DATASOURCE to NOTE
dtypes: category(1), int32(3), string(3)

In [252]:
#vehicles = vehicles.rename(columns={"LASTUPDATE": "LASTUPDATE_V"})

In [253]:
# vehicles.head(3)

,DATASOURCE,DAYOFSERVICE,VEHICLEID,DISTANCE,MINUTES,LASTUPDATE_V,NOTE
0,DB,23-NOV-18 00:00:00,3303848,286166,58849,04-DEC-18 08:03:09,<NA>
1,DB,23-NOV-18 00:00:00,3303847,259545,56828,04-DEC-18 08:03:09,<NA>
2,DB,28-FEB-18 00:00:00,2868329,103096,40967,08-MAR-18 10:35:59,<NA>


In [254]:
# print(type(vehicles))

<class 'dask.dataframe.core.DataFrame'>


##### work backwards bit.
 - none of the columns really matter. keeping vehicle ID just incase but expect it to be useless. its in leacvetimes anyway

## Merging

In [255]:
#vehicles.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 7 entries, DATASOURCE to NOTE
dtypes: category(1), int32(3), string(3)

In [94]:
trips.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 7 entries, DAYOFSERVICE to ACTUALTIME_DEP
dtypes: category(4), float64(2), int32(1)

In [95]:
leaveTimes

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID
0,01-JAN-18 00:00:00,5972116,12,119,48030.0,48030.0,48012.0,48012.0,2693211
1,01-JAN-18 00:00:00,5966674,12,119,54001.0,54001.0,54023.0,54023.0,2693267
2,01-JAN-18 00:00:00,5959105,12,119,60001.0,60001.0,59955.0,59955.0,2693263
3,01-JAN-18 00:00:00,5966888,12,119,58801.0,58801.0,58771.0,58771.0,2693284
4,01-JAN-18 00:00:00,5965960,12,119,56401.0,56401.0,56309.0,56323.0,2693209
...,...,...,...,...,...,...,...,...,...
664446,31-DEC-18 00:00:00,8588153,78,4383,28605.0,28605.0,28998.0,29013.0,3265721
664447,31-DEC-18 00:00:00,8587459,78,4383,22695.0,22695.0,23247.0,23247.0,3265687
664448,31-DEC-18 00:00:00,8586183,78,4383,51481.0,51481.0,52237.0,52283.0,2693229
664449,31-DEC-18 00:00:00,8589374,23,7053,53659.0,53659.0,53525.0,53525.0,3265669


In [96]:
leaveTimes.head(3)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID
0,01-JAN-18 00:00:00,5972116,12,119,48030.0,48030.0,48012.0,48012.0,2693211
1,01-JAN-18 00:00:00,5966674,12,119,54001.0,54001.0,54023.0,54023.0,2693267
2,01-JAN-18 00:00:00,5959105,12,119,60001.0,60001.0,59955.0,59955.0,2693263


In [97]:
leaveTimes.shape

(116949113, 9)

In [ ]:
trips= trips.compute()

- merge needed to add the route id and line id

In [98]:
merged_dd = leaveTimes.merge(trips, how="left", on=["TRIPID", "DAYOFSERVICE"])
merged_dd.head(50)

TypeError: Can only merge Series or DataFrame objects, a <class 'dask.dataframe.core.DataFrame'> was passed

In [260]:
#merged_dd = merged_dd.persist()

- tried to merge on tripid but there is a data issue because the same id was sometimes reused, resulting in the creation of 600million more rows 


In [261]:
vehicles.head(3)


,DATASOURCE,DAYOFSERVICE,VEHICLEID,DISTANCE,MINUTES,LASTUPDATE_V,NOTE
0,DB,23-NOV-18 00:00:00,3303848,286166,58849,04-DEC-18 08:03:09,<NA>
1,DB,23-NOV-18 00:00:00,3303847,259545,56828,04-DEC-18 08:03:09,<NA>
2,DB,28-FEB-18 00:00:00,2868329,103096,40967,08-MAR-18 10:35:59,<NA>


In [262]:
# print(len(vehicles))
# print(len(leaveTimes)) 116949113 rows
# print(len(trips))

In [263]:
merged_dd.info()
#len(merged_dd) 
#116949113 rows

<class 'dask.dataframe.core.DataFrame'>
Columns: 14 entries, DAYOFSERVICE to ACTUALTIME_DEP_T
dtypes: category(1), category(1), category(1), object(1), float64(6), int32(4)

- too many rows means many dont match

In [264]:
# vehicles['DAYOFSERVICE'] = vehicles.DAYOFSERVICE.astype('category')
# had made a mistake earlier in assigning categories. 
# migh go back and change later bu t dont want to wait for code to run now

In [265]:
# merged_dd = merged_dd.persist()

In [266]:
merged_dd

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_DEP_T,ACTUALTIME_DEP_T
npartitions=176,,,,,,,,,,,,,,
,object,int32,int32,int32,float64,float64,float64,float64,int32,category[known],category[known],category[known],float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [267]:
# len(merged_dd) # 116949113

In [268]:
#merged_dd.memory_usage

In [269]:
merged_dd.head(50)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_DEP_T,ACTUALTIME_DEP_T
0,01-JAN-18 00:00:00,5972116,12,119,48030.0,48030.0,48012.0,48012.0,2693211,1,1_37,1,47400.0,47427.0
1,01-JAN-18 00:00:00,5966674,12,119,54001.0,54001.0,54023.0,54023.0,2693267,1,1_37,1,53400.0,53410.0
2,01-JAN-18 00:00:00,5959105,12,119,60001.0,60001.0,59955.0,59955.0,2693263,1,1_37,1,59400.0,59426.0
3,01-JAN-18 00:00:00,5966888,12,119,58801.0,58801.0,58771.0,58771.0,2693284,1,1_37,1,58200.0,58220.0
4,01-JAN-18 00:00:00,5965960,12,119,56401.0,56401.0,56309.0,56323.0,2693209,1,1_37,1,55800.0,55807.0
5,01-JAN-18 00:00:00,5965964,12,119,70688.0,70688.0,70663.0,70679.0,2693209,1,1_37,1,70200.0,70236.0
6,01-JAN-18 00:00:00,5958117,12,119,72488.0,72488.0,72539.0,72539.0,2172293,1,1_37,1,72000.0,72023.0
7,01-JAN-18 00:00:00,5959109,12,119,74288.0,74288.0,74173.0,74173.0,2693263,1,1_37,1,73800.0,73806.0
8,01-JAN-18 00:00:00,5972114,12,119,40187.0,40187.0,40096.0,40111.0,2693211,1,1_37,1,39600.0,39681.0
9,01-JAN-18 00:00:00,5959099,12,119,38387.0,38387.0,38271.0,38271.0,2693263,1,1_37,1,37800.0,37810.0


In [270]:
test_tripid = merged_dd.loc[merged_dd["TRIPID"] == 5972116]


In [271]:
test_tripid.head(80)

/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/dask/dataframe/core.py:7506: UserWarning: Insufficient elements for `head`. 80 elements requested, only 42 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_DEP_T,ACTUALTIME_DEP_T
0,01-JAN-18 00:00:00,5972116,12,119,48030.0,48030.0,48012.0,48012.0,2693211,1,1_37,1,47400.0,47427.0
29,01-JAN-18 00:00:00,5972116,13,44,48079.0,48079.0,48058.0,48058.0,2693211,1,1_37,1,47400.0,47427.0
57,01-JAN-18 00:00:00,5972116,14,7603,48114.0,48114.0,48071.0,48071.0,2693211,1,1_37,1,47400.0,47427.0
232,01-JAN-18 00:00:00,5972116,15,45,48155.0,48155.0,48089.0,48100.0,2693211,1,1_37,1,47400.0,47427.0
261,01-JAN-18 00:00:00,5972116,16,46,48221.0,48221.0,48137.0,48137.0,2693211,1,1_37,1,47400.0,47427.0
289,01-JAN-18 00:00:00,5972116,17,47,48300.0,48300.0,48171.0,48171.0,2693211,1,1_37,1,47400.0,47427.0
471,01-JAN-18 00:00:00,5972116,18,48,48366.0,48366.0,48189.0,48189.0,2693211,1,1_37,1,47400.0,47427.0
499,01-JAN-18 00:00:00,5972116,19,49,48460.0,48460.0,48276.0,48276.0,2693211,1,1_37,1,47400.0,47427.0
709,01-JAN-18 00:00:00,5972116,21,52,48610.0,48610.0,48358.0,48378.0,2693211,1,1_37,1,47400.0,47427.0
737,01-JAN-18 00:00:00,5972116,22,265,48744.0,48744.0,48463.0,48463.0,2693211,1,1_37,1,47400.0,47427.0


- latest update column all at wrong day so completely useless
- prognumber is each stop it was just at, beginning at starting point of route
    - no real point of keeping end point of route
- some questions of reliability of the data because I found instances when the reported arrival time in the trips data was earlier than some of the stops arrival times
- date seems insignificant so will be converting it to days of the week to give bigger volume of data for models
- vehicle stats seem insignificantas they only refer to the stats of the vehicle on the day and dont give context to how many times the drivers ahve changed


##### kernel could not compute so i had to go backwards and remove them before entering


In [272]:
# merged_dd = merged_dd.drop({"LASTUPDATE_V", "MINUTES", "DISTANCE", "ACTUALTIME_ARR_T", "PLANNEDTIME_ARR_T", "LASTUPDATE"}, axis=1).compute()


In [273]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8759 entries, 0 to 8758
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   dt            8759 non-null   int64         
 1   temp          8759 non-null   int64         
 2   visibility    8759 non-null   int64         
 3   wind_speed    8759 non-null   int64         
 4   weather_main  8759 non-null   int64         
 5   date          8759 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(5)
memory usage: 410.7 KB


In [274]:
weather.head(5)

,dt,temp,visibility,wind_speed,weather_main,date
0,1514768400,2,10,6,1,2018-01-01 01:00:00
1,1514772000,2,10,6,0,2018-01-01 02:00:00
2,1514775600,2,10,6,0,2018-01-01 03:00:00
3,1514779200,2,10,6,0,2018-01-01 04:00:00
4,1514782800,2,10,5,0,2018-01-01 05:00:00


In [275]:
weather['temp'] = weather['temp'].astype('int8')
weather['visibility'] = weather['visibility'].astype('int8')
weather['wind_speed'] = weather['wind_speed'].astype('int8')
#weather['weather_description'] = weather['weather_description'].astype('int8')
weather['weather_main'] = weather['weather_main'].astype('int8')




In [276]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8759 entries, 0 to 8758
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   dt            8759 non-null   int64         
 1   temp          8759 non-null   int8          
 2   visibility    8759 non-null   int8          
 3   wind_speed    8759 non-null   int8          
 4   weather_main  8759 non-null   int8          
 5   date          8759 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), int8(4)
memory usage: 171.2 KB


In [277]:
merged_dd.head(3)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_DEP_T,ACTUALTIME_DEP_T
0,01-JAN-18 00:00:00,5972116,12,119,48030.0,48030.0,48012.0,48012.0,2693211,1,1_37,1,47400.0,47427.0
1,01-JAN-18 00:00:00,5966674,12,119,54001.0,54001.0,54023.0,54023.0,2693267,1,1_37,1,53400.0,53410.0
2,01-JAN-18 00:00:00,5959105,12,119,60001.0,60001.0,59955.0,59955.0,2693263,1,1_37,1,59400.0,59426.0


In [278]:
merged_dd.dtypes

DAYOFSERVICE           object
TRIPID                  int32
PROGRNUMBER             int32
STOPPOINTID             int32
PLANNEDTIME_ARR       float64
PLANNEDTIME_DEP       float64
ACTUALTIME_ARR        float64
ACTUALTIME_DEP        float64
VEHICLEID               int32
LINEID               category
ROUTEID              category
DIRECTION            category
PLANNEDTIME_DEP_T     float64
ACTUALTIME_DEP_T      float64
dtype: object

In [279]:
merged_dd['PROGRNUMBER'] = merged_dd['PROGRNUMBER'].astype('int8')

In [280]:
merged_dd['DIRECTION'] = merged_dd['DIRECTION'].astype('int8')

In [281]:
# merged_dd = merged_dd.persist()

In [282]:
merged_dd.dtypes

DAYOFSERVICE           object
TRIPID                  int32
PROGRNUMBER              int8
STOPPOINTID             int32
PLANNEDTIME_ARR       float64
PLANNEDTIME_DEP       float64
ACTUALTIME_ARR        float64
ACTUALTIME_DEP        float64
VEHICLEID               int32
LINEID               category
ROUTEID              category
DIRECTION                int8
PLANNEDTIME_DEP_T     float64
ACTUALTIME_DEP_T      float64
dtype: object

In [283]:
merged_dd.head(3)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_DEP_T,ACTUALTIME_DEP_T
0,01-JAN-18 00:00:00,5972116,12,119,48030.0,48030.0,48012.0,48012.0,2693211,1,1_37,1,47400.0,47427.0
1,01-JAN-18 00:00:00,5966674,12,119,54001.0,54001.0,54023.0,54023.0,2693267,1,1_37,1,53400.0,53410.0
2,01-JAN-18 00:00:00,5959105,12,119,60001.0,60001.0,59955.0,59955.0,2693263,1,1_37,1,59400.0,59426.0


###### remove nan values from buses that never left
- cant change the NaN values in dask so must do it in pandas when in individual files

In [284]:
# merged_dd["PLANNEDTIME_DEP_T"] = merged_dd["PLANNEDTIME_DEP_T"].replace(regex={"NaN": 0})
merged_dd["PLANNEDTIME_DEP_T"] = merged_dd["PLANNEDTIME_DEP_T"].astype('int32').fillna(0)

In [285]:
merged_dd["PLANNEDTIME_ARR"] = merged_dd["PLANNEDTIME_ARR"].astype('int32').fillna(0)

In [286]:
merged_dd["PLANNEDTIME_DEP"] = merged_dd["PLANNEDTIME_DEP"].astype('int32').fillna(0)

In [287]:
merged_dd.dtypes

DAYOFSERVICE           object
TRIPID                  int32
PROGRNUMBER              int8
STOPPOINTID             int32
PLANNEDTIME_ARR         int32
PLANNEDTIME_DEP         int32
ACTUALTIME_ARR        float64
ACTUALTIME_DEP        float64
VEHICLEID               int32
LINEID               category
ROUTEID              category
DIRECTION                int8
PLANNEDTIME_DEP_T       int32
ACTUALTIME_DEP_T      float64
dtype: object

In [288]:
merged_dd.head(3)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_DEP_T,ACTUALTIME_DEP_T
0,01-JAN-18 00:00:00,5972116,12,119,48030,48030,48012.0,48012.0,2693211,1,1_37,1,47400,47427.0
1,01-JAN-18 00:00:00,5966674,12,119,54001,54001,54023.0,54023.0,2693267,1,1_37,1,53400,53410.0
2,01-JAN-18 00:00:00,5959105,12,119,60001,60001,59955.0,59955.0,2693263,1,1_37,1,59400,59426.0


In [289]:
#merged_dd = merged_dd.persist()

In [290]:
def date_from_epoch(x):
    y = dt.datetime.fromtimestamp (x).strftime('%Y-%m-%d')
    return y   

weather['DAYOFSERVICE'] = weather['dt'].apply(lambda x: date_from_epoch(x))

def round_epoch_hour(x):
    day = x // 86400
    day = day * 86400
    time = x - day
    round_time = time // 3600
    return round_time

weather['HOUR_DEPARTURE'] = weather['dt'].apply(lambda x: round_epoch_hour(x))

In [291]:
weather.head(5)

,dt,temp,visibility,wind_speed,weather_main,date,DAYOFSERVICE,HOUR_DEPARTURE
0,1514768400,2,10,6,1,2018-01-01 01:00:00,2018-01-01,1
1,1514772000,2,10,6,0,2018-01-01 02:00:00,2018-01-01,2
2,1514775600,2,10,6,0,2018-01-01 03:00:00,2018-01-01,3
3,1514779200,2,10,6,0,2018-01-01 04:00:00,2018-01-01,4
4,1514782800,2,10,5,0,2018-01-01 05:00:00,2018-01-01,5


In [292]:
merged_dd['HOUR_DEPARTURE'] = merged_dd['ACTUALTIME_DEP'].apply(lambda row: row//3600)

/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/dask/dataframe/core.py:3974: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('ACTUALTIME_DEP', 'float64'))

  warnings.warn(meta_warning(meta))


In [293]:
merged_dd.head(5)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_DEP_T,ACTUALTIME_DEP_T,HOUR_DEPARTURE
0,01-JAN-18 00:00:00,5972116,12,119,48030,48030,48012.0,48012.0,2693211,1,1_37,1,47400,47427.0,13.0
1,01-JAN-18 00:00:00,5966674,12,119,54001,54001,54023.0,54023.0,2693267,1,1_37,1,53400,53410.0,15.0
2,01-JAN-18 00:00:00,5959105,12,119,60001,60001,59955.0,59955.0,2693263,1,1_37,1,59400,59426.0,16.0
3,01-JAN-18 00:00:00,5966888,12,119,58801,58801,58771.0,58771.0,2693284,1,1_37,1,58200,58220.0,16.0
4,01-JAN-18 00:00:00,5965960,12,119,56401,56401,56309.0,56323.0,2693209,1,1_37,1,55800,55807.0,15.0


In [294]:
merged_dd.dtypes

DAYOFSERVICE           object
TRIPID                  int32
PROGRNUMBER              int8
STOPPOINTID             int32
PLANNEDTIME_ARR         int32
PLANNEDTIME_DEP         int32
ACTUALTIME_ARR        float64
ACTUALTIME_DEP        float64
VEHICLEID               int32
LINEID               category
ROUTEID              category
DIRECTION                int8
PLANNEDTIME_DEP_T       int32
ACTUALTIME_DEP_T      float64
HOUR_DEPARTURE        float64
dtype: object

In [295]:
weather.dtypes

dt                         int64
temp                        int8
visibility                  int8
wind_speed                  int8
weather_main                int8
date              datetime64[ns]
DAYOFSERVICE              object
HOUR_DEPARTURE             int64
dtype: object

In [296]:
merged_dd["DAYOFSERVICE"] = merged_dd["DAYOFSERVICE"].astype("datetime64")

In [297]:
#change day of service to match 
merged_dd["DAYOFSERVICE"] = merged_dd["DAYOFSERVICE"].dt.strftime('%Y-%m-%d')

In [298]:
merged_dd["DAYOFSERVICE"] = merged_dd["DAYOFSERVICE"].astype("datetime64")
merged_dd['DAYOFWEEK'] = merged_dd['DAYOFSERVICE'].dt.weekday

In [299]:
merged_dd

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_DEP_T,ACTUALTIME_DEP_T,HOUR_DEPARTURE,DAYOFWEEK
npartitions=176,,,,,,,,,,,,,,,,
,datetime64[ns],int32,int8,int32,int32,int32,float64,float64,int32,category[known],category[known],int8,int32,float64,float64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [300]:
merged_dd.head(5)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_DEP_T,ACTUALTIME_DEP_T,HOUR_DEPARTURE,DAYOFWEEK
0,2018-01-01,5972116,12,119,48030,48030,48012.0,48012.0,2693211,1,1_37,1,47400,47427.0,13.0,0
1,2018-01-01,5966674,12,119,54001,54001,54023.0,54023.0,2693267,1,1_37,1,53400,53410.0,15.0,0
2,2018-01-01,5959105,12,119,60001,60001,59955.0,59955.0,2693263,1,1_37,1,59400,59426.0,16.0,0
3,2018-01-01,5966888,12,119,58801,58801,58771.0,58771.0,2693284,1,1_37,1,58200,58220.0,16.0,0
4,2018-01-01,5965960,12,119,56401,56401,56309.0,56323.0,2693209,1,1_37,1,55800,55807.0,15.0,0


In [301]:
check = merged_dd.loc[merged_dd['DAYOFWEEK'] != 0]

In [302]:
check.head(5)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_DEP_T,ACTUALTIME_DEP_T,HOUR_DEPARTURE,DAYOFWEEK
157744,2018-01-02,5958030,59,4385,51599,51599,51331.0,51349.0,2172293,15,15_17,2,47400,47400.0,14.0,1
157745,2018-01-02,5967345,59,4385,55199,55199,54902.0,54920.0,2868404,15,15_17,2,51000,51000.0,15.0,1
157746,2018-01-02,5965972,59,4385,47999,47999,47612.0,47612.0,2693250,15,15_17,2,43800,43777.0,13.0,1
157747,2018-01-02,5957141,59,4385,50999,50999,50140.0,50152.0,2693284,15,15_17,2,46800,46887.0,13.0,1
157748,2018-01-02,5967112,59,4385,44399,44399,44292.0,44304.0,2693282,15,15_17,2,40200,40203.0,12.0,1


In [303]:
merged_dd.dtypes

DAYOFSERVICE         datetime64[ns]
TRIPID                        int32
PROGRNUMBER                    int8
STOPPOINTID                   int32
PLANNEDTIME_ARR               int32
PLANNEDTIME_DEP               int32
ACTUALTIME_ARR              float64
ACTUALTIME_DEP              float64
VEHICLEID                     int32
LINEID                     category
ROUTEID                    category
DIRECTION                      int8
PLANNEDTIME_DEP_T             int32
ACTUALTIME_DEP_T            float64
HOUR_DEPARTURE              float64
DAYOFWEEK                     int64
dtype: object

In [307]:
weather.dtypes

dt                         int64
temp                        int8
visibility                  int8
wind_speed                  int8
weather_main                int8
date              datetime64[ns]
DAYOFSERVICE              object
HOUR_DEPARTURE             int64
dtype: object

In [308]:
weather["DAYOFSERVICE"] = weather["DAYOFSERVICE"].astype("datetime64")

In [309]:
merged_dd.head(3)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_DEP_T,ACTUALTIME_DEP_T,HOUR_DEPARTURE,DAYOFWEEK
0,2018-01-01,5972116,12,119,48030,48030,48012.0,48012.0,2693211,1,1_37,1,47400,47427.0,13.0,0
1,2018-01-01,5966674,12,119,54001,54001,54023.0,54023.0,2693267,1,1_37,1,53400,53410.0,15.0,0
2,2018-01-01,5959105,12,119,60001,60001,59955.0,59955.0,2693263,1,1_37,1,59400,59426.0,16.0,0


In [310]:
merged_dd = dd.merge(merged_dd, weather, on=['DAYOFSERVICE', 'HOUR_DEPARTURE'], how='left')


In [311]:
merged_dd.head(5)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,...,PLANNEDTIME_DEP_T,ACTUALTIME_DEP_T,HOUR_DEPARTURE,DAYOFWEEK,dt,temp,visibility,wind_speed,weather_main,date
0,2018-01-01,5972116,12,119,48030,48030,48012.0,48012.0,2693211,1,...,47400,47427.0,13.0,0,1.514812e+09,2.0,7.0,6.0,1.0,2018-01-01 13:00:00
1,2018-01-01,5966674,12,119,54001,54001,54023.0,54023.0,2693267,1,...,53400,53410.0,15.0,0,1.514819e+09,2.0,10.0,6.0,1.0,2018-01-01 15:00:00
2,2018-01-01,5959105,12,119,60001,60001,59955.0,59955.0,2693263,1,...,59400,59426.0,16.0,0,1.514822e+09,2.0,10.0,5.0,0.0,2018-01-01 16:00:00
3,2018-01-01,5966888,12,119,58801,58801,58771.0,58771.0,2693284,1,...,58200,58220.0,16.0,0,1.514822e+09,2.0,10.0,5.0,0.0,2018-01-01 16:00:00
4,2018-01-01,5965960,12,119,56401,56401,56309.0,56323.0,2693209,1,...,55800,55807.0,15.0,0,1.514819e+09,2.0,10.0,6.0,1.0,2018-01-01 15:00:00


In [315]:
#merged_dd = merged_dd.persist()

In [316]:
merged_dd.dtypes

DAYOFSERVICE         datetime64[ns]
TRIPID                        int32
PROGRNUMBER                    int8
STOPPOINTID                   int32
PLANNEDTIME_ARR               int32
PLANNEDTIME_DEP               int32
ACTUALTIME_ARR              float64
ACTUALTIME_DEP              float64
VEHICLEID                     int32
LINEID                     category
ROUTEID                    category
DIRECTION                      int8
PLANNEDTIME_DEP_T             int32
ACTUALTIME_DEP_T            float64
HOUR_DEPARTURE              float64
DAYOFWEEK                     int64
dt                            int64
temp                           int8
visibility                     int8
wind_speed                     int8
weather_main                   int8
date                 datetime64[ns]
dtype: object

In [317]:
merged_dd['ROUTEID'] = merged_dd['ROUTEID'].astype(str)

In [318]:
merged_dd['ROUTE_DIRECTION'] = merged_dd['ROUTEID'] + '_' + merged_dd['DIRECTION'].astype(str)

In [319]:
merged_dd.dtypes

DAYOFSERVICE         datetime64[ns]
TRIPID                        int32
PROGRNUMBER                    int8
STOPPOINTID                   int32
PLANNEDTIME_ARR               int32
PLANNEDTIME_DEP               int32
ACTUALTIME_ARR              float64
ACTUALTIME_DEP              float64
VEHICLEID                     int32
LINEID                     category
ROUTEID                      object
DIRECTION                      int8
PLANNEDTIME_DEP_T             int32
ACTUALTIME_DEP_T            float64
HOUR_DEPARTURE              float64
DAYOFWEEK                     int64
dt                            int64
temp                           int8
visibility                     int8
wind_speed                     int8
weather_main                   int8
date                 datetime64[ns]
ROUTE_DIRECTION              object
dtype: object

In [320]:
merged_dd['ROUTEID'] = merged_dd['ROUTEID'].astype("category")

In [321]:
merged_dd['ROUTE_DIRECTION'] = merged_dd['ROUTE_DIRECTION'].astype("category")

In [322]:
# merged_dd = merged_dd.persist()

In [323]:
fortySixA1 = merged_dd.loc[merged_dd['ROUTE_DIRECTION']=="46A_68_1"]

In [324]:
fortySixA1

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_DEP_T,ACTUALTIME_DEP_T,HOUR_DEPARTURE,DAYOFWEEK,dt,temp,visibility,wind_speed,weather_main,date,ROUTE_DIRECTION
npartitions=176,,,,,,,,,,,,,,,,,,,,,,,
,datetime64[ns],int32,int8,int32,int32,int32,float64,float64,int32,category[known],category[unknown],int8,int32,float64,float64,int64,int64,int8,int8,int8,int8,datetime64[ns],category[unknown]
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [325]:
merged_dd

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_DEP_T,ACTUALTIME_DEP_T,HOUR_DEPARTURE,DAYOFWEEK,dt,temp,visibility,wind_speed,weather_main,date,ROUTE_DIRECTION
npartitions=176,,,,,,,,,,,,,,,,,,,,,,,
,datetime64[ns],int32,int8,int32,int32,int32,float64,float64,int32,category[known],category[unknown],int8,int32,float64,float64,int64,int64,int8,int8,int8,int8,datetime64[ns],category[unknown]
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


# make the task number smaller
- starting on 9506 for fortysixa1 and 8978 for merged_dd 

In [312]:
#fortySixA1 = fortySixA1.persist()

In [313]:
#print(len(fortySixA1)) 14474

In [314]:
print(type(fortySixA1))

NameError: name 'fortySixA1' is not defined

- converts to pandas

In [ ]:
%%time
#fortySixA1_pd = fortySixA1.compute()

In [ ]:
print(type(fortySixA1_pd))

In [ ]:
fortySixA1_pd.to_csv('./test_46a_dask.csv', index=False)  

In [ ]:
# merged_dd = pd.merge_asof(merged_dd, weather, on="col_name", direction='nearest')

In [ ]:
#merged_dd['epoch'] = merged_dd.DAYOFSERVICE.values.astype(np.int64).compute()

In [ ]:
#merged_dd['DAYOFSERVICE'] = merged_dd.DAYOFSERVICE.dt.day_name()
#merged_dd['DAYOFSERVICE'] = pd.factorize(merged_dd['DAYOFSERVICE'], sort=True)[0] # wasn't right


# idea is to create an extra column that rounds to the most recent hour in epoch form and join weather on that 

In [ ]:
merged_dd.dtypes

In [ ]:
weather.dtypes

In [ ]:
# safety cell to make sure code below doens't run
printsdfgdsdg

### code for when ready to export file
- must sort the weather join first

In [ ]:
routes = merged_dd.ROUTEID.unique()
routes = sorted(routes)
print(len(routes))
print(routes)

In [ ]:
lines = merged_dd.LINEID.unique()
lines = sorted(lines)
print(lines)

- pretty sure lines is the correct one to use but could look into it further

In [ ]:
directions = [1, 2]
#for d in directions:
    for i in routes:
        temp_rows = merged_dd.loc[(merged_dd['LINEID'] == i) & (merged_dd['DIRECTION'] == d)]
        name = f"line_{i}_direction{d}"
        
        
        temp_rows.to_csv(f'.{}/{name}.csv')
        
        
#for d in directions:
    for r in routes:
        temp_rows = merged_dd.loc[(merged_dd['ROUTEID'] == r) & (merged_dd['DIRECTION'] == d)]
        
        line_folder = temp_rows["LINEID"]
        
        
        name = f"route_{r}_direction{d}"
        
        
        temp_rows.to_csv(f'.{line_folder}/{name}.csv')
        

        
    
    